# ノートブックの概要
- 特徴量は入力データそのまま
- モデルはlightGBM
- パラメータチューニングなし

# 前処理

In [ ]:
import polars as pl
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_cols(100)
import numpy as np

import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
%matplotlib inline

from datetime import date

from prophet import Prophet

from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# データ読み込みから前処理まで

# データを読み込む
train = pl.read_csv("../data/input/train.csv")
test = pl.read_csv("../data/input/test.csv")
sample_submit = pl.read_csv("../data/input/sample_submit.csv", has_header = False)

# IDと日時を分散する
train = train.insert_column(0, train["datetime"].alias("id")).with_columns(pl.col("datetime").str.strptime(dtype = pl.Date))
test = test.insert_column(0, test["datetime"].alias("id")).with_columns(pl.col("datetime").str.strptime(dtype = pl.Date))

# 休業日(close = 1)(と、お盆だけど休業していなくて結局引っ越し数0だった日)を分離する
train = train.filter(
    (pl.col("close") != 1)
    & (pl.col("id") != "2010-08-18")
    & (pl.col("id") != "2011-08-14")
)
test_close = test.filter(pl.col("close") == 1)[["id"]]
test_close = test_close.with_columns(pl.Series("y", [0.0] * len(test_close)))
test = test.filter(pl.col("close") != 1)

# 目的変数を対数変換する
train = train.insert_column(3, train["y"].log().alias("y_ln"))

# closeをカラムごと削除
train = train.drop("close")
test = test.drop("close")

# 2010年は料金区分に関する情報が欠損しているので学習から削除
train = train.filter(pl.col("datetime") >= date(2011, 1, 1))

print(train.head(5))

# モデル構築

## prophetによるベースライン作成

In [ ]:
# polarsのdatetime型をpandasのdatetime型に変換する必要がある
# 訓練データ
train_pandas = (
    train
    .select(["datetime", "y_ln"])
    .rename({"datetime": "ds", "y_ln": "y"})
    .to_pandas()
)
# テストデータ
test_pandas = (
    test
    .select(["datetime"])
    .rename({"datetime": "ds"})
    .to_pandas()
)
# 学習
model = Prophet()
model.fit(train_pandas)

# 予測
forecast_train = model.predict(train_pandas)
forecast_test = model.predict(test_pandas)

# 予測結果の可視化
fig, ax = plt.subplots(
    figsize=(12, 5),
    constrained_layout=True,
)
ax.set_title("Prophetによる予測（Train=青, Test=赤）(縦軸：y_ln、横軸：datetime)")

# train側は Prophet の標準描画（青・点群付き）
model.plot(model.predict(train_pandas), ax=ax)

# test側は後から重ね描き（赤）
ax.plot(
    forecast_test["ds"], forecast_test["yhat"],
    color="red", linewidth=2, label="Test yhat"
)
ax.fill_between(
    forecast_test["ds"],
    forecast_test["yhat_lower"], forecast_test["yhat_upper"],
    color="red", alpha=0.2, label="Test interval"
) 

ax.legend()
plt.show()

上手い感じにトレンドを捉えている

In [ ]:
# 推定値の算出
forecast_train_pl = pl.Series("y_ln_prophet", forecast_train["yhat"].values)
forecast_test_pl = pl.Series("y_ln_prophet", forecast_test["yhat"].values)

# 推定値をDataFrameにまとめる
train = train.insert_column(4, forecast_train_pl)
test = test.insert_column(3, forecast_test_pl)

# 残差列を追加
train = train.with_columns(
    (train["y_ln"] - train["y_ln_prophet"]).alias("y_ln_difference")
)

## 特徴量追加

In [ ]:
# 特徴量追加なし

## 学習と予測

In [ ]:
# 機械学習ライブラリと親和性の高いPandasに変換する
train_pandas = train.to_pandas().set_index("id")

# 目的変数と特徴量
target_column = "y_ln_difference"
feature_columns = ["client", "price_am", "price_pm"]

# 今回は時系列予測なので、訓練データと評価データを分割する際に、時系列の順番を考慮する必要がある
# 今回は、2015年1月1日を境に訓練データと評価データを分割する
# 訓練データと検証データを期間で分割
train_temp = train_pandas[train_pandas["datetime"] < "2015-01-01"]
valid_temp = train_pandas[train_pandas["datetime"] >= "2015-01-01"]

# 説明変数と目的変数を分割
X_train, y_train = train_temp[feature_columns], train_temp[target_column]
X_valid, y_valid = valid_temp[feature_columns], valid_temp[target_column]

# 学習
model = GradientBoostingRegressor(random_state = 1192).fit(X_train, y_train)

# 予測(モデルの予測値をベースラインであるy_ln_prophetに足した後、対数変換を逆変換)
y_pred_train = np.exp(model.predict(X_train) + train_temp["y_ln_prophet"])
y_pred_valid = np.exp(model.predict(X_valid) + valid_temp["y_ln_prophet"])

In [ ]:
# 可視化
fig, axes = plt.subplots(
    nrows=3, ncols=2,
    height_ratios=[1, 1, 1], width_ratios=[2, 1],
    figsize=(15, 12)
)

g_time_train = sns.lineplot(data = train_pandas, x="datetime", y="y", ax=axes[0, 0])
g_predict_train = sns.lineplot(data = train_temp, x="datetime", y=y_pred_train, ax=axes[0, 0])
g_predict_valid = sns.lineplot(data = valid_temp, x="datetime", y=y_pred_valid, ax=axes[0, 0])

g_time_hist_train = sns.histplot(data = train_pandas, x="y", bins=50, ax=axes[0, 1])
g_predict_hist_train = sns.histplot(data = train_temp, x=y_pred_train, bins=50, ax=axes[0, 1])
g_predict_hist_valid = sns.histplot(data = valid_temp, x=y_pred_valid, bins=50, ax=axes[0, 1])

# 残差の確認
g_diff_train = sns.scatterplot(data = train_temp, x="datetime", y=train_temp["y"] - y_pred_train, marker = "+", ax=axes[1, 0])
g_diff_valid = sns.scatterplot(data = valid_temp, x="datetime", y=valid_temp["y"] - y_pred_valid, marker = "+", ax=axes[1, 0])
axes[1, 0].set_ylabel("残差")

g_hist_diff_train = sns.histplot(data = train_temp, x=train_temp["y"] - y_pred_train, bins=50, ax=axes[1, 1])
g_hist_diff_valid = sns.histplot(data = valid_temp, x=valid_temp["y"] - y_pred_valid, bins=50, ax=axes[1, 1])

# 特徴量重要度
g_importance = sns.barplot(x=model.feature_importances_, y=feature_columns, ax=axes[2, 0])
g_importance.set_title("特徴量重要度")

axes[2, 1].axis("off")

plt.tight_layout()
plt.show()

# 予測精度の確認
print("訓練データの評価関数:", np.round(mean_absolute_error(train_temp["y"], y_pred_train), decimals = 7))# 小数第7位はコンペスコアの有効数字
print("評価データの評価関数:", np.round(mean_absolute_error(valid_temp["y"], y_pred_valid), decimals = 7))

- 下部が上手く予測出来ていない
  - 何によるものなのかは不明だが、日時に関する特徴量を何も渡していないのでまともに予測できるはずもないだろう。

上昇傾向は上手く捉えられている(prophetのおかげかな)  
実際、残差の分布については何とか揃えられている  
引っ越し数yが低い場所が上手く予測できていない。何だか、下限が一つの曲線で抑えられてしまっている？  
改めて、やっぱ特徴量が少なすぎる。もっといろいろ増やそう

## 提出データ出力

In [ ]:
# # 提出前に、全ての訓練データを使って学習させておく
# X, y = train_pandas[feature_columns], train_pandas[target_column]
# model = GradientBoostingRegressor(random_state = 1192).fit(X, y)

# # 機械学習ライブラリと親和性の高いPandasに変換する
# test_pandas = test.to_pandas().set_index("id")

# # 説明変数の分割
# X_test = test_pandas[feature_columns]

# # 予測
# y_pred_test = np.exp(model.predict(X_test) + test_pandas["y_ln_prophet"])

# # 提出用DataFrame
# submit = pl.DataFrame({
#     "id": X_test.index.values,
#     "y": y_pred_test.values
# })

# # 退避していた休業日のデータを結合
# submit = pl.concat(
#     [submit, test_close],
#     how = "vertical_relaxed"
# ).sort("id")

# # 提出ファイルを保存する
# # submit.write_csv("../data/output/submit_original_features_gbt_default_prm.csv", include_header = False)

In [ ]:
# # テストデータに対する予測の可視化
# fig, axes = plt.subplots(
#     nrows=1, ncols=2,
#     height_ratios=[1], width_ratios=[2, 1],
#     figsize=(15, 4)
# )

# g_time_train = sns.lineplot(data = train_pandas, x="datetime", y="y", ax=axes[0])
# g_time_test = sns.lineplot(data = test_pandas, x="datetime", y=y_pred_test, ax=axes[0])

# g_hist_train = sns.histplot(data = train_pandas, x="y", ax=axes[1])
# g_hist_test = sns.histplot(data = test_pandas, x=y_pred_test, ax=axes[1])

# 参考サイト  
[コンペサイト アップル 引越し需要予測](https://signate.jp/competitions/269/data)  
[SIGNATE SOTA アップル 引越し需要予測 備忘録](https://zenn.dev/tremendous1192/articles/ea6e73359ee764)  


# htmlに変換したものを出力

In [ ]:
# from datetime import datetime
# from pathlib import Path

# base_name = "model_特徴量追加なし"
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# output_dir = Path("html")
# output_file = f"{base_name}_{timestamp}"

# !jupyter nbconvert --to html EDA.ipynb --output "{output_file}" --output-dir "{output_dir}"